# Get Live Box Scores (including Players)

In [121]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import sqlite3
import seaborn as sns
from matplotlib.pyplot import figure
from bs4 import BeautifulSoup
import time
import requests     
import shutil       
import datetime
from scipy.stats import norm
import warnings
warnings.filterwarnings('ignore')
import requests
import json
import xgboost
from xgboost import XGBClassifier
from random import randint
import  random
import os
import plotly.graph_objs as go
os.chdir('C:\\Users\\Travis\\OneDrive\\Data Science\\Personal_Projects\\Sports\\NBA_Prediction_V3_1')
from cmath import nan
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion
from sklearn.tree import plot_tree
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.compose import make_column_selector as selector, ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
import pickle
from sklearn.metrics import fbeta_score
import winsound
from sklearn.linear_model import LinearRegression
from sklearn import tree, preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, plot_confusion_matrix, recall_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, ExtraTreesClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_curve, auc, f1_score, make_scorer, recall_score
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from matplotlib.offsetbox import OffsetImage, AnnotationBbox

In [122]:
yahoo_sports = 'https://sports.yahoo.com/nba/scoreboard/'
driver = None
if driver == None:
    driver = webdriver.Chrome()
driver.get(yahoo_sports)
soup = BeautifulSoup(driver.page_source, 'html.parser')
# get links in scoreboard
links = soup.find_all('a')

# append all links
all_links = []
for link in links:
    all_links.append(link.get('href'))

all_links

# only keep if starts with /nba/
all_links = [link for link in all_links if link != None]
all_links = [link for link in all_links if link.startswith('/nba/')]
all_links = pd.DataFrame(all_links, columns = ['link'])

all_links['full_link'] = 'https://sports.yahoo.com' + all_links['link']
all_links




,link,full_link
0,/nba/golden-state-warriors-boston-celtics-2023...,https://sports.yahoo.com/nba/golden-state-warr...
1,/nba/toronto-raptors-minnesota-timberwolves-20...,https://sports.yahoo.com/nba/toronto-raptors-m...
2,/nba/brooklyn-nets-phoenix-suns-2023011921/,https://sports.yahoo.com/nba/brooklyn-nets-pho...
3,/nba/philadelphia-76ers-portland-trail-blazers...,https://sports.yahoo.com/nba/philadelphia-76er...
4,/nba/chicago-bulls-detroit-pistons-2023011908/,https://sports.yahoo.com/nba/chicago-bulls-det...


In [123]:
def get_all_player_data(url):
    driver.get(url)
    # WAIT FOR PAGE TO LOAD
    time.sleep(2)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    match_stats = soup.find('div', class_ = 'match-stats')
    time.sleep(2)
    # find all tables   
    try:
        tables = match_stats.find_all('table')
        # read table with pandas
        away_starters = pd.read_html(str(tables[0]))[0]
        away_starters['Starters'] = away_starters['Starters'].str[:-1]
        # rename Starters to Players
        away_starters = away_starters.rename(columns = {'Starters': 'Players'})

        home_starters = pd.read_html(str(tables[1]))[0]
        home_starters['Starters'] = home_starters['Starters'].str[:-1]
        # rename Starters to Players
        home_starters = home_starters.rename(columns = {'Starters': 'Players'})

        away_bench = pd.read_html(str(tables[2]))[0]
        # rename Bench to Players
        away_bench = away_bench.rename(columns = {'Bench': 'Players'})

        home_bench = pd.read_html(str(tables[3]))[0]
        # rename Bench to Players
        home_bench = home_bench.rename(columns = {'Bench': 'Players'})
        # append all together
        all_starters = away_starters.append(home_starters)
        all_bench = away_bench.append(home_bench)
        # append all together
        all_players = all_starters.append(all_bench)
        # reset index
        all_players = all_players.reset_index(drop = True)
        # drop na
        all_players = all_players.dropna(subset = ['Min'])
        # drop any min column without a : in it
        all_players = all_players[all_players['Min'].str.contains(':')]
        # Pts to int
        all_players['Pts'] = all_players['Pts'].astype(int)
        # Sort by Pts
        all_players = all_players.sort_values(by = 'Pts', ascending = False)
        

        return all_players
    except:
        pass

In [66]:
# get game 1
game_1 = get_all_player_data(all_links['full_link'][0])
game_1

,Players,Min,Reb,Ast,Stl,Blk,Pts
8,Jalen Green,39:39,5,7,0,0,41
3,Terry Rozier,38:13,2,6,2,1,26
7,Alperen Sengun,39:14,12,6,2,1,24
2,Mason Plumlee,29:18,9,1,3,0,17
12,Mark Williams,18:42,6,0,0,5,17
0,P.J. Washington,37:17,8,2,3,3,16
4,LaMelo Ball,20:29,1,4,0,0,13
1,Jalen McDaniels,34:54,4,3,5,1,12
9,Eric Gordon,30:59,2,2,1,1,11
5,Kenyon Martin Jr.,32:18,7,3,0,0,11


In [84]:
game_2 = get_all_player_data(all_links['full_link'][1])
game_2

,Players,Min,Reb,Ast,Stl,Blk,Pts
5,Lauri Markkanen,30:56,10,1,0,2,34
3,Norman Powell,29:56,4,3,0,0,30
9,Mike Conley,26:21,3,8,0,0,17
2,Ivica Zubac,26:00,12,1,0,2,15
6,Malik Beasley,27:08,4,0,0,0,13
7,Walker Kessler,26:03,11,3,1,2,13
8,Jordan Clarkson,28:41,2,4,0,0,12
0,Nicolas Batum,23:18,5,1,0,0,11
23,Collin Sexton,19:30,1,1,1,1,11
11,Reggie Jackson,19:03,2,2,0,0,11


In [85]:
game_3 = get_all_player_data(all_links['full_link'][2])
game_3

,Players,Min,Reb,Ast,Stl,Blk,Pts
7,Nikola Jokic,14:03,5,6,1,0,19
23,Bruce Brown,11:01,2,2,0,0,10
0,Jaden McDaniels,15:40,3,1,0,2,9
1,Kyle Anderson,12:24,3,2,3,1,9
2,Naz Reid,15:09,2,1,0,1,7
10,Taurean Prince,12:30,3,0,0,1,6
3,Anthony Edwards,16:14,5,0,0,0,6
12,Luka Garza,5:08,1,0,0,0,5
8,Kentavious Caldwell-Pope,13:13,0,1,3,0,5
25,Bones Hyland,6:34,0,0,0,0,4


In [71]:
driver = None
if driver == None:
    driver = webdriver.Chrome()

### Lakers Vs Kings
- Lebron hits 35
- Harrison Barnes 19
- De'Aaron Fox 24
- INDVIDUAL bet: Westbrook hits 20

- Bet 3: Lebron hits 37 AND Nuggets Win

In [120]:

lakers_kings = get_all_player_data(all_links['full_link'][3])
lakers_kings

,Players,Min,Reb,Ast,Stl,Blk,Pts
5,LeBron James,36:30,8,9,1,1,32
3,De'Aaron Fox,39:07,3,6,1,0,31
0,Harrison Barnes,40:08,6,2,0,0,20
23,Russell Westbrook,31:53,7,5,1,0,19
4,Kevin Huerter,32:55,3,8,2,1,18
2,Richaun Holmes,30:50,11,1,0,0,16
26,Kendrick Nunn,16:28,1,0,0,1,14
24,Max Christie,27:09,3,1,2,0,12
1,Keegan Murray,36:49,10,0,0,0,10
25,Wenyen Gabriel,23:33,6,1,1,1,10
